<a href="https://colab.research.google.com/github/sunnatillo91/ai-datascience/blob/main/flat_border_uysotV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Python script to convert SVG files to PNG images:

## 1. Parse the Coordinates

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the DataFrame
df = pd.read_csv("/content/drive/MyDrive/full_flatspoint.csv", header=0)

# Function to convert coordinate string to list of tuples
def parse_coordinates(coord_str):
    coords = coord_str.split()
    return [(float(coord.split(',')[0]), float(coord.split(',')[1])) for coord in coords]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42565 entries, 0 to 42564
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             42565 non-null  int64 
 1   image_point    42560 non-null  object
 2   absolute_path  42342 non-null  object
dtypes: int64(1), object(2)
memory usage: 997.7+ KB


In [ ]:
print(f"Mavjud bo'lmagan qiymatlar soni: {df.isnull().sum().sum()}")

Mavjud bo'lmagan qiymatlar soni: 228


In [ ]:
df

,id,image_point,absolute_path
0,157397,"975,338.74 975,741.51 998.94,741.51 998.94,770...",/v1/public/files/image/240720-105350-840ab749-...
1,157395,"1492.87,591.91 1700.02,591.91 1700.02,785.11 1...",/v1/public/files/image/240720-112219-26fd8330-...
2,157394,"1604.02,331.91 1604.02,516.89 1710.49,516.89 1...",/v1/public/files/image/240720-112219-26fd8330-...
3,157393,"1395.94,434.13 1493.72,434.13 1493.72,516.89 1...",/v1/public/files/image/240720-112219-26fd8330-...
4,157392,"1286.66,294.89 1385.85,294.89 1385.85,516.89 1...",/v1/public/files/image/240720-112219-26fd8330-...
...,...,...,...
42560,7927,"179,423 179,549.8 273.6,549.8 278.9,554.8 325....",/v1/public/files/image/220718-160248-ec71250e-...
42561,7926,"434.1,461.4 434.1,549.8 385.3,549.8 380,554.8 ...",/v1/public/files/image/220718-160248-ec71250e-...
42562,7925,"442.9,549.8 492,549.9 497,554.8 543.1,554.8 54...",/v1/public/files/image/220718-160248-ec71250e-...
42563,5830,"625.338623046875,178.38345336914062 657.135620...",/v1/public/files/image/240525-120646-06fe45c2-...


In [ ]:
non_standard_count = 0

for i, k in enumerate(df['image_point']):
    try:
        parse_coordinates(k)
    except Exception as e:
        df.drop(index=i, inplace=True)
        non_standard_count += 1

print("Not standard rows deleted, total:", non_standard_count)

Not standard rows deleted, total: 61


In [ ]:
df = df.dropna()

In [ ]:
print(df.columns)

Index(['id', 'image_point', 'absolute_path'], dtype='object')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42282 entries, 0 to 42564
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             42282 non-null  int64 
 1   image_point    42282 non-null  object
 2   absolute_path  42282 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [ ]:
df.loc[:, 'coordinates'] = df['image_point'].apply(parse_coordinates)

In [ ]:
df['coordinates'][:5]

,coordinates
0,"[(975.0, 338.74), (975.0, 741.51), (998.94, 74..."
1,"[(1492.87, 591.91), (1700.02, 591.91), (1700.0..."
2,"[(1604.02, 331.91), (1604.02, 516.89), (1710.4..."
3,"[(1395.94, 434.13), (1493.72, 434.13), (1493.7..."
4,"[(1286.66, 294.89), (1385.85, 294.89), (1385.8..."


In [ ]:
def add_full_path(string):
  new_str = f"https://service.ahouse.softex.uz{string}"
  return new_str

In [ ]:
df.loc[:, 'absolute_path'] = df['absolute_path'].apply(add_full_path)

In [ ]:
df[['absolute_path']].to_csv('/content/drive/MyDrive/uysot2/path.csv')

##2. Draw the Coordinates on an Image
Here, we use OpenCV to draw the coordinates on a blank image and save it as a PNG file.

In [ ]:
!pip install opencv-python numpy

In [ ]:
import os
import numpy as np
import cv2
import requests

def draw_coordinates_on_image(coordinates, img_width, img_height, output_path):
    image = np.ones((img_height, img_width, 3), np.uint8) * 255
    points = [(int(x), int(y)) for x, y in coordinates]
    points = np.array(points, np.int32)
    points = points.reshape((-1, 1, 2))
    cv2.polylines(image, [points], isClosed=True, color=(0, 0, 0), thickness=2)
    cv2.imwrite(output_path, image)

def convert_to_yolo_format(coords, orig_width, orig_height, img_width, img_height):
    scale_x = img_width / orig_width
    scale_y = img_height / orig_height

    scaled_coords = [(coord[0] * scale_x, coord[1] * scale_y) for coord in coords]

    min_x = max(min(coord[0] for coord in scaled_coords), 0)
    max_x = min(max(coord[0] for coord in scaled_coords), img_width)
    min_y = max(min(coord[1] for coord in scaled_coords), 0)
    max_y = min(max(coord[1] for coord in scaled_coords), img_height)

    x_center = (min_x + max_x) / 2 / img_width
    y_center = (min_y + max_y) / 2 / img_height
    width = (max_x - min_x) / img_width
    height = (max_y - min_y) / img_height

    # Validate coordinates are between 0 and 1
    if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
        print(f"Debug info: min_x={min_x}, max_x={max_x}, min_y={min_y}, max_y={max_y}")
        print(f"Debug info: x_center={x_center}, y_center={y_center}, width={width}, height={height}")
        raise ValueError(f"Non-normalized or out of bounds coordinates: {x_center}, {y_center}, {width}, {height}")

    return f'0 {x_center} {y_center} {width} {height}'



In [ ]:
# Directory paths
output_image_dir = '/content/drive/MyDrive/uysot2/images'
output_label_dir = '/content/drive/MyDrive/uysot2/labels'

# Ensure the directories exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Create subdirectories for train and val
os.makedirs(os.path.join(output_image_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(output_image_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(output_label_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(output_label_dir, 'val'), exist_ok=True)

# Split data into train and val
train_ratio = 0.8
train_size = int(len(df) * train_ratio)

orig_width, orig_height = 1920, 1080
resized_width, resized_height = 640, 640

for i, row in df.iterrows():
    coordinates = row['coordinates']
    if i < train_size:
        image_subdir = 'train'
    else:
        image_subdir = 'val'

    # Fetch and save the image
    output_image_path = os.path.join(output_image_dir, image_subdir, f'image_{i}.png')
    try:
        response = requests.get(row['absolute_path'], timeout=10)
        response.raise_for_status()
        with open(output_image_path, 'wb') as f:
            f.write(response.content)
        draw_coordinates_on_image(coordinates, resized_width, resized_height, output_image_path)

        # Convert coordinates to YOLO format and save the label
        try:
            yolo_label = convert_to_yolo_format(coordinates, orig_width, orig_height, resized_width, resized_height)
            output_label_path = os.path.join(output_label_dir, image_subdir, f'image_{i}.txt')
            with open(output_label_path, 'w') as f:
                f.write(yolo_label)
        except ValueError as e:
            print(f"Skipping image {output_image_path}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {row['absolute_path']}: {e}")


## Move 20 % of images in train folder to val

In [ ]:
# import os
# import shutil
# import random

# # Define the paths
# train_folder = '/content/drive/MyDrive/uysot2/images/train'
# val_folder = '/content/drive/MyDrive/uysot2/images/val'

# # Get a list of all images in the train folder
# train_images = os.listdir(train_folder)

# # Calculate the number of images to move (20% of the total images)
# num_images_to_move = int(len(train_images) * 0.20)

# # Randomly select images to move
# images_to_move = random.sample(train_images, num_images_to_move)

# # Move the selected images
# for image in images_to_move:
#     shutil.move(os.path.join(train_folder, image), os.path.join(val_folder, image))

# print(f"Moved {num_images_to_move} images from train to val folder.")

Moved 5115 images from train to val folder.


## Deleting the folder if it's no need

In [ ]:
# import shutil
# import os

# directory_path = '/content/drive/MyDrive/uysot22'

# # Check if the directory exists
# if os.path.exists(directory_path):
#     try:
#         # Delete the directory and its contents
#         shutil.rmtree(directory_path)
#         print(f'{directory_path} and its contents have been deleted.')
#     except Exception as e:
#         print(f'Error: {directory_path} cannot be deleted. {e}')
# else:
#     print(f'{directory_path} does not exist.')

/content/drive/MyDrive/uysot2 and its contents have been deleted.


## 2. Validate

In [ ]:
# Function to validate and convert data
# def validate_data(label_folder, resized_width, resized_height):
#     invalid_files = []

#     for label_file in os.listdir(label_folder):
#         if label_file.endswith(".txt"):
#             with open(os.path.join(label_folder, label_file), "r") as file:
#                 coords = [tuple(map(float, line.split())) for line in file]

#                 try:
#                     yolo_format = convert_to_yolo_format(coords, resized_width, resized_height)
#                 except ValueError as e:
#                     print(f"Bounding box out of image bounds in {label_file}: {e}")
#                     invalid_files.append(label_file)

#     if invalid_files:
#         print(f"Invalid files: {', '.join(invalid_files)}")

# # Example usage
# validate_data('/content/drive/MyDrive/uysot2/labels/train', 640, 640)

NameError: name 'convert_to_yolo_format' is not defined

In [ ]:
!pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.3/682.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0


##3. Configure the YOLO Model
Create a YAML configuration file for your dataset. This file should specify paths to the training and validation datasets and the number of classes.

###`Clone yolov5 model`

In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git /content/drive/MyDrive/uysot2/yolov5

%cd /content/drive/MyDrive/uysot2/yolov5

# Install the required dependencies
!pip install requests==2.31.0
!pip install -qr requirements.txt comet_ml  # install

Cloning into '/content/drive/MyDrive/uysot3/yolov5'...
remote: Enumerating objects: 16836, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 16836 (delta 1), reused 6 (delta 0), pack-reused 16825
Receiving objects: 100% (16836/16836), 15.57 MiB | 10.07 MiB/s, done.
Resolving deltas: 100% (11550/11550), done.
Updating files: 100% (148/148), done.
/content/drive/MyDrive/uysot3/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all th

In [ ]:
!pip install requests==2.31.0

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3


In [ ]:
import yaml

# Define dataset configuration
dataset_config = {
    'train': '/content/drive/MyDrive/uysot2/images/train',
    'val': '/content/drive/MyDrive/uysot2/images/val',
    'nc': 1,  # number of classes
    'names': ['room_border']
}

# Save to dataset.yaml
with open('/content/drive/MyDrive/uysot2/dataset.yaml', 'w') as f:
    yaml.dump(dataset_config, f)

## 4. Train
YOLO requires annotations in a specific text format where each line corresponds to one object in the image. Each line has the following format: class_id x_center y_center width height.

Here's how to convert your coordinates into YOLO format

## Number of images

In [ ]:
import shutil
import os
print(len(os.listdir('/content/drive/MyDrive/uysot/images/train')))

print(len(os.listdir('/content/drive/MyDrive/uysot/images/val')))

1594
403


# Train the YOLOv5 model
!python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/uysot2/dataset.yaml --cfg models/yolov5m.yaml --weights yolov5m.pt --name room_border_detector

In [ ]:
!rm -rf /content/drive/MyDrive/uysot2/images/train.cache

In [ ]:
!pip install --upgrade pillow

  Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.5 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.0.1
    Uninstalling Pillow-9.0.1:
      Successfully uninstalled Pillow-9.0.1


In [ ]:
!pip install --upgrade pillow==10.4.0 thop==0.1.1.post2209072238 requests==2.31.0
!pip install -U albumentations

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3


In [ ]:
# Train YOLOv5s on dataset for 50 epochs
%cd /content/drive/MyDrive/uysot2/yolov5

!python /content/drive/MyDrive/uysot2/yolov5/train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/uysot2/dataset.yaml --weights yolov5m.pt --evolve 300 --cache

/content/drive/MyDrive/uysot3/yolov5
2024-08-06 15:16:28.412418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 15:16:28.431827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 15:16:28.437925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=/content/drive/MyDrive/uysot3/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=300, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, imag

# Image prediction

In [ ]:
# Restart runtime
import os
os._exit(00)

# Install necessary packages
!pip install ultralytics==8.2.73
!pip install gitpython
!pip install pillow>=10.3.0 requests>=2.32.0 thop>=0.1.1

# Navigate to YOLOv5 directory
%cd /content/drive/MyDrive/uysot2/yolov5

# Run the detection script
!python detect.py --weights /content/drive/MyDrive/uysot2/model/best.pt --img 640 --conf 0.25 --source /content/drive/MyDrive/uysot2/test_images --project /content/drive/MyDrive/uysot2 --name predictions --save-txt --save-conf

## Image size check

In [ ]:
from PIL import Image

def get_image_size(image_path):
    image = Image.open(image_path)
    width, height = image.size
    return width, height

image_path = '/content/drive/MyDrive/uysot2/321.jpg'
width, height = get_image_size(image_path)
print(f"Width: {width}, Height: {height}")


Width: 1920, Height: 1080


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
!import ultralytics

!python /content/drive/MyDrive/uysot2/yolov5/detect.py --weights /content/drive/MyDrive/uysot2/model/best.pt --img 640 --conf 0.25 --source /content/drive/MyDrive/uysot2/test_images --project /content/drive/MyDrive/uysot2 --name predictions --save-txt --save-conf

## Save the YOLO model to /content/drive/MyDrive/uysot2

In [ ]:
import shutil
import os

# Define source and destination paths
source_dir = '/content/drive/MyDrive/uysot2/yolov5/runs/train/exp3/weights'
dest_dir = '/content/drive/MyDrive/uysot2/model'

# Ensure the destination directory exists
os.makedirs(dest_dir, exist_ok=True)

# Copy all files from the source to the destination directory
for filename in os.listdir(source_dir):
    full_file_name = os.path.join(source_dir, filename)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, dest_dir)

print("Model saved to:", dest_dir)


# Draw images according to predicted coordinates

In [ ]:
import os
import cv2

# Directory paths
image_dir = '/content/drive/MyDrive/uysot2/test_images'
label_dir = '/content/drive/MyDrive/uysot2/predictions6/labels'
output_dir = '/content/drive/MyDrive/uysot2/predictions6/overlay'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

def draw_predictions_on_image(img_path, label_path, output_path):
    # Load the image
    image = cv2.imread(img_path)
    img_height, img_width, _ = image.shape

    # Load the labels if available
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            labels = f.readlines()

        for label in labels:
            # Extract the bounding box coordinates
            parts = label.strip().split()
            if len(parts) != 5:
                print(f"Skipping line in {label_path} due to unexpected format: {label}")
                continue

            try:
                x_center, y_center, width, height = map(float, parts[1:])
            except ValueError:
                print(f"Error converting values in {label_path} for line: {label}")
                continue

            # Convert YOLO format to pixel coordinates
            min_x = int((x_center - width / 2) * img_width)
            min_y = int((y_center - height / 2) * img_height)
            max_x = int((x_center + width / 2) * img_width)
            max_y = int((y_center + height / 2) * img_height)

            # Draw bounding box
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
    else:
        print(f"No label file found for {img_path}")

    # Save the output image with predictions drawn
    cv2.imwrite(output_path, image)

# Iterate over all images and labels
for image_file in os.listdir(image_dir):
    if image_file.endswith('.png') or image_file.endswith('.jpg'):
        img_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, image_file.replace('.png', '.txt').replace('.jpg', '.txt'))
        output_path = os.path.join(output_dir, image_file)

        draw_predictions_on_image(img_path, label_path, output_path)

print("Predictions drawn and saved.")


Skipping line in /content/drive/MyDrive/uysot/predictions6/labels/240720-081230-9e049641-44ac-482f.txt due to unexpected format: 0 0.421354 0.243981 0.154167 0.230556 0.285285

Skipping line in /content/drive/MyDrive/uysot/predictions6/labels/240720-081230-9e049641-44ac-482f.txt due to unexpected format: 0 0.585417 0.241667 0.15625 0.231481 0.513635

Skipping line in /content/drive/MyDrive/uysot/predictions6/labels/240720-081230-9e049641-44ac-482f.txt due to unexpected format: 0 0.269531 0.776852 0.146354 0.355556 0.786637

Skipping line in /content/drive/MyDrive/uysot/predictions6/labels/240720-081230-9e049641-44ac-482f.txt due to unexpected format: 0 0.109635 0.74213 0.167187 0.341667 0.821214

Skipping line in /content/drive/MyDrive/uysot/predictions6/labels/photo_2024-07-27_12-09-31.txt due to unexpected format: 0 0.585156 0.240972 0.15625 0.234722 0.508353

Skipping line in /content/drive/MyDrive/uysot/predictions6/labels/photo_2024-07-27_12-09-31.txt due to unexpected format: 0 0